<a href="https://colab.research.google.com/github/arkmarcos/Teste_Drive/blob/main/Ler_PDF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf2 pytesseract Pillow pdf2image

!apt-get install tesseract-ocr poppler-utils

!wget https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata

!mv por.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

import PyPDF2
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import io

pdf_file_path = 'Book.pdf'

with open(pdf_file_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)

    all_text = ""

    for page_num in range(num_pages):
        # Tente extrair o texto diretamente do PDF (se possível)
        page_text = pdf_reader.pages[page_num].extract_text()
        if page_text:
            all_text += page_text + "\n"
            continue

        # Caso a extração direta falhe, converta a página em imagem e processe o OCR
        images = convert_from_path(pdf_file_path, first_page=page_num+1, last_page=page_num+1, dpi=200)  # Ajuste o DPI se necessário
        img = images[0]

        # Processar o OCR na memória sem salvar em arquivo
        with io.BytesIO() as output:
            img.save(output, format='PNG')
            output.seek(0)
            page_text = pytesseract.image_to_string(Image.open(output), lang='eng', config='--psm 6 --oem 1')

        all_text += page_text + "\n"

print(all_text)

In [ ]:
!pip install -q -U google-generativeai
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('Google_Api_Key')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(f"Traduza o texto a seguir para o português do Brasil: {all_text}")
print(response.text)